This example shows accessing of Coreweave's s3 object storage using the boto3 library.

Restart your kernal after running the following block

In [ ]:
%pip install boto3

In [ ]:
import os
import boto3

#defining the credentials
ACCESS_KEY = 'your_access_key'
SECRET_KEY = 'your_secret_key'
REGION_NAME = 'ord1'
ENDPOINT_URL = 'http://object.ord1.coreweave.com'
BUCKET_NAME = 'ml-datasets'

#creating the client
client = boto3.client("s3", aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name=REGION_NAME, endpoint_url=ENDPOINT_URL)

In [ ]:
#listing all the buckets
response = client.list_buckets()
print("Listing S3 Buckets:")
for bucket in response['Buckets']:
    print(f"-- {bucket['Name']}")

In [ ]:
#listing all files in a bucket

objects = client.list_objects_v2(Bucket=BUCKET_NAME)

for obj in objects['Contents']:
    print(obj['Key'])

In [ ]:
# List objects at the top level of the bucket
response = client.list_objects_v2(Bucket=BUCKET_NAME, Delimiter='/')

# Iterate through the top-level objects
for obj in response.get('Contents', []):
    print(f'Top-level object: {obj["Key"]}')
#Iterate through the top-level directories
for common_prefix in response.get('CommonPrefixes', []):
    print(f'Top-level folder: {common_prefix["Prefix"]}')

In [ ]:
# Upload a directory to the bucket

local_directory = './example_notebook'
# Walk through the local directory and upload each file
for root, dirs, files in os.walk(local_directory):
    for file in files:
        local_path = os.path.join(root, file)
        s3_path = os.path.relpath(local_path, local_directory)
        client.upload_file(local_path, BUCKET_NAME, s3_path)
        print(f'Uploaded {local_path} to S3://{BUCKET_NAME}/{s3_path}')

In [ ]:
# Download a directory from the bucket to local system
s3_directory = 'Author/Albert_Bushnell_Hart'
local_directory = './Albert_Bushnell_Hart'

# List objects in the S3 directory
objects = client.list_objects_v2(Bucket='ml-datasets', Prefix=s3_directory)

# Iterate through objects and download each one
for obj in objects.get('Contents', []):
    s3_path = obj['Key']
    local_path = os.path.join(local_directory, os.path.relpath(s3_path, s3_directory))
    os.makedirs(os.path.dirname(local_path), exist_ok=True)
    client.download_file(BUCKET_NAME, s3_path, local_path)
    print(f'Downloaded S3://{BUCKET_NAME}/{s3_path} to {local_path}')

In [ ]:
# Set the bucket name and directory path to delete
directory_path = 'folderToDelete'

# List objects in the directory
objects = client.list_objects_v2(Bucket=BUCKET_NAME, Prefix=directory_path)

# Delete each object (file) in the directory
for obj in objects.get('Contents', []):
    client.delete_object(Bucket=BUCKET_NAME, Key=obj['Key'])
    print(f'Deleted: S3://{BUCKET_NAME}/{obj["Key"]}')

# Delete the directory itself
client.delete_object(Bucket=BUCKET_NAME, Key=directory_path)
print(f'Deleted directory: S3://{BUCKET_NAME}/{directory_path}')